In [1]:
import pandas as pd
import sqlite3
from pymongo import MongoClient
from pathlib import Path

def upload_data_to_db(csv_file_path, database_type):
    # Create a Path object
    csv_file = Path(csv_file_path)

    # Check if the file exists
    if not csv_file.exists():
        print("The specified CSV file does not exist.")
        return

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    filename = csv_file.stem  # Get the filename without the extension

    if database_type.lower() == 'sqlite':
        # Connect to SQLite database (or create it)
        conn = sqlite3.connect('local_database.db')
        cursor = conn.cursor()

        # Create a table with the filename
        df.to_sql(filename, conn, if_exists='replace', index=False)
        print(f"Data from {csv_file} has been added to the SQLite database as table '{filename}'.")

        # Commit changes and close the connection
        conn.commit()
        conn.close()

    elif database_type.lower() == 'mongodb':
        # Connect to MongoDB (local)
        client = MongoClient('mongodb://localhost:27017/')
        db = client['local_database']  # Specify the database name

        # Upload data to MongoDB collection
        data = df.to_dict(orient='records')
        db[filename].insert_many(data)  # Insert data into collection named after the filename
        print(f"Data from {csv_file} has been uploaded to MongoDB collection '{filename}'.")

    else:
        print("Unsupported database type. Please use 'SQLite' or 'MongoDB'.")

In [2]:
csv_file_path = 'courses.csv'  # Replace with your CSV file path
database_type = 'mongodb' #'sqlite'  # or 'mongodb'
upload_data_to_db(csv_file_path, database_type)


Data from courses.csv has been added to the SQLite database as table 'courses'.


In [4]:
def get_sqlite_data(db_path, sql_query):
    """
    Connects to SQLite and retrieves data based on the SQL query.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(sql_query)
    data = cursor.fetchall()
    conn.close()
    return data

In [5]:
sql_query = """
SELECT * 
FROM courses;
"""

In [6]:
get_sqlite_data("local_database.db", sql_query)

[(101, 'Data Structures', 2, 'Dr. Brown', 3),
 (102, 'Calculus', 3, 'Dr. Smith', 4),
 (103, 'Database Systems', 2, 'Dr. Brown', 3),
 (104, 'Linear Algebra', 3, 'Dr. Smith', 3),
 (105, 'Introduction to Business', 4, 'Dr. White', 3)]